In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn import metrics, preprocessing, model_selection
pd.options.display.max_columns = 100

In [2]:
train_df = pd.read_csv("../input/train_cm.csv")
test_df = pd.read_csv("../input/test_cm.csv")
print(train_df.shape, test_df.shape)

(341424, 24) (146765, 23)


In [3]:
train_df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,05/04/18,05/04/18,06/04/18,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,7.706428
1,03930f033646d073462b35d411616323597715ac4fc398...,23/01/15,11/04/15,16/04/15,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,6.662563
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,28/01/15,01/02/15,05/02/15,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,7.871602
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,02/05/15,11/06/15,16/06/15,1,1,2,2,46,2,2,3,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,5.344943
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,02/09/15,14/12/15,19/12/15,1,1,2,0,46,2,2,4,5,2.0,7.0,2,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,D,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,7.059346


In [4]:
date_cols = ["booking_date", "checkin_date", "checkout_date"]
for date_col in date_cols:
    train_df[date_col] = pd.to_datetime(train_df[date_col], format="%d/%m/%y")
    test_df[date_col] = pd.to_datetime(test_df[date_col], format="%d/%m/%y")
    
    train_df[date_col + "_in_seconds"] = (train_df[date_col] - dt.datetime(1970,1,1)).dt.total_seconds()
    test_df[date_col + "_in_seconds"] = (test_df[date_col] - dt.datetime(1970,1,1)).dt.total_seconds()
    
    train_df[date_col + "_month"] = train_df[date_col].dt.month
    test_df[date_col + "_month"] = test_df[date_col].dt.month
    
    train_df[date_col + "_year"] = train_df[date_col].dt.year
    test_df[date_col + "_year"] = test_df[date_col].dt.year
    
    train_df[date_col + "_week"] = train_df[date_col].dt.year
    test_df[date_col + "_week"] = test_df[date_col].dt.year
    
train_df["days_stay"] = (train_df["checkout_date"] - train_df["checkin_date"]).dt.days
test_df["days_stay"] = (test_df["checkout_date"] - test_df["checkin_date"]).dt.days

train_df["days_advance_booking"] = (train_df["checkin_date"] - train_df["booking_date"]).dt.days
test_df["days_advance_booking"] = (test_df["checkin_date"] - test_df["booking_date"]).dt.days

In [5]:
train_df = train_df.drop("amount_spent_per_room_night_scaled", axis=1)

In [6]:
all_df = pd.concat([train_df, test_df]).reset_index(drop=True)
all_df = all_df.sort_values(by="checkin_date").reset_index(drop=True)
print(all_df.shape)

(488189, 37)


In [7]:
additional_cols = ["memberid", "resort_id", "state_code_residence", "checkin_date", "booking_date"]
new_df = all_df[["reservation_id"] + additional_cols] 

In [8]:
### get the mean of important columns
for col in ["booking_date_in_seconds", "checkin_date_in_seconds", 
            "days_stay", "days_advance_booking", "roomnights",
           ]:
    gdf = all_df.groupby("memberid")[col].mean().reset_index()
    gdf.columns = ["memberid", "member_"+col+"_mean"]
    new_df = pd.merge(new_df, gdf, on="memberid", how="left")
    

    
### get the mean of important columns
for col in [ 
            "days_stay", "roomnights",
           ]:
    gdf = all_df.groupby("memberid")[col].sum().reset_index()
    gdf.columns = ["memberid", "member_"+col+"_sum"]
    new_df = pd.merge(new_df, gdf, on="memberid", how="left")
    
### get the mean of important columns
for col in [ 
            "resort_id",
           ]:
    gdf = all_df.groupby("memberid")[col].nunique().reset_index()
    gdf.columns = ["memberid", "member_"+col+"_nunique"]
    new_df = pd.merge(new_df, gdf, on="memberid", how="left")
    


In [10]:
new_df["member_cumnum_of_booking"] = all_df.groupby("memberid")["booking_date_in_seconds"].cumcount().values
new_df["cumsum_member_days_stay"] = all_df.groupby("memberid")["days_stay"].cumsum().values
new_df["cumsum_member_total_pax"] = all_df.groupby("memberid")["total_pax"].cumsum().values

all_df["prev_booking_date"] = all_df.groupby("memberid")["booking_date_in_seconds"].shift(1)
new_df["time_gap_booking_prev"] = all_df["booking_date_in_seconds"] - all_df["prev_booking_date"]

all_df["next_booking_date"] = all_df.groupby("memberid")["booking_date_in_seconds"].shift(-1)
new_df["time_gap_booking_next"] = all_df["booking_date_in_seconds"] - all_df["next_booking_date"]

all_df["prev_checkin_date"] = all_df.groupby("memberid")["checkin_date_in_seconds"].shift(1)
new_df["time_gap_checkin_prev"] = all_df["checkin_date_in_seconds"] - all_df["prev_checkin_date"]

all_df["next_checkin_date"] = all_df.groupby("memberid")["checkin_date_in_seconds"].shift(-1)
new_df["time_gap_checkin_next"] = all_df["checkin_date_in_seconds"] - all_df["next_checkin_date"]

all_df["prev_checkout_date"] = all_df.groupby("memberid")["checkout_date_in_seconds"].shift(1)
new_df["time_gap_checkout_prev"] = all_df["checkout_date_in_seconds"] - all_df["prev_checkout_date"]

all_df["next_checkout_date"] = all_df.groupby("memberid")["checkout_date_in_seconds"].shift(-1)
new_df["time_gap_checkout_next"] = all_df["checkout_date_in_seconds"] - all_df["next_checkout_date"]

In [11]:
all_df["prev_resort_booking_date"] = all_df.groupby(["memberid", "resort_id"])["booking_date_in_seconds"].shift(1)
new_df["time_gap_booking_prev_resort"] = all_df["booking_date_in_seconds"] - all_df["prev_resort_booking_date"]

all_df["next_resort_booking_date"] = all_df.groupby(["memberid", "resort_id"])["booking_date_in_seconds"].shift(-1)
new_df["time_gap_booking_next_resort"] = all_df["booking_date_in_seconds"] - all_df["next_resort_booking_date"]

all_df["prev_resort_checkin_date"] = all_df.groupby(["memberid", "resort_id"])["checkin_date_in_seconds"].shift(1)
new_df["time_gap_checkin_prev_resort"] = all_df["checkin_date_in_seconds"] - all_df["prev_resort_checkin_date"]

all_df["next_resort_checkin_date"] = all_df.groupby(["memberid", "resort_id"])["checkin_date_in_seconds"].shift(-1)
new_df["time_gap_checkin_next_resort"] = all_df["checkin_date_in_seconds"] - all_df["next_resort_checkin_date"]

In [12]:
all_df["prev2_booking_date"] = all_df.groupby("memberid")["booking_date_in_seconds"].shift(2)
new_df["time_gap_booking_prev2"] = all_df["booking_date_in_seconds"] - all_df["prev2_booking_date"]

all_df["next2_booking_date"] = all_df.groupby("memberid")["booking_date_in_seconds"].shift(-2)
new_df["time_gap_booking_next2"] = all_df["booking_date_in_seconds"] - all_df["next2_booking_date"]

all_df["prev2_checkin_date"] = all_df.groupby("memberid")["checkin_date_in_seconds"].shift(2)
new_df["time_gap_checkin_prev2"] = all_df["checkin_date_in_seconds"] - all_df["prev2_checkin_date"]

all_df["next2_checkin_date"] = all_df.groupby("memberid")["checkin_date_in_seconds"].shift(-2)
new_df["time_gap_checkin_next2"] = all_df["checkin_date_in_seconds"] - all_df["next2_checkin_date"]

In [13]:
### Info on prev and next visits
for col in ["days_stay", "roomnights", "days_advance_booking"]:
    all_df["prev_"+col] = all_df.groupby("memberid")[col].shift(1)
    new_df["prev_diff_"+col] = all_df[col] - all_df["prev_"+col]
    
    all_df["next_"+col] = all_df.groupby("memberid")[col].shift(-1)
    new_df["next_diff_"+col] = all_df[col] - all_df["next_"+col]

In [14]:
### Info on prev and next visits
for col in ["channel_code", "room_type_booked_code", "resort_type_code", "main_product_code"]:
    all_df["prev_"+col] = all_df.groupby("memberid")[col].shift(1)
    new_df["prev_diff_"+col] = (all_df[col] == all_df["prev_"+col]).astype(int)
    
    all_df["next_"+col] = all_df.groupby("memberid")[col].shift(-1)
    new_df["next_diff_"+col] = (all_df[col] == all_df["next_"+col]).astype(int)

In [15]:
### pivot on member and resort
gdf = pd.pivot_table(all_df, index="memberid", columns="resort_id", values="reservation_id", aggfunc="count", fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="memberid", how="left")

gdf = pd.pivot_table(all_df, index="memberid", columns="checkin_date_year", values="reservation_id", aggfunc="count", fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="memberid", how="left")

gdf = pd.pivot_table(all_df, index="memberid", columns="resort_type_code", values="reservation_id", aggfunc="count", fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="memberid", how="left")

gdf = pd.pivot_table(all_df, index="memberid", columns="room_type_booked_code", values="reservation_id", aggfunc="count", fill_value=0).reset_index()
new_df = pd.merge(new_df, gdf, on="memberid", how="left")


In [18]:
new_df = new_df.drop(additional_cols, axis=1)
new_df.head()

,reservation_id,member_booking_date_in_seconds_mean,member_checkin_date_in_seconds_mean,member_days_stay_mean,member_days_advance_booking_mean,member_roomnights_mean,member_days_stay_sum,member_roomnights_sum,member_resort_id_nunique,member_cumnum_of_booking,cumsum_member_days_stay,cumsum_member_total_pax,time_gap_booking_prev,time_gap_booking_next,time_gap_checkin_prev,time_gap_checkin_next,time_gap_checkout_prev,time_gap_checkout_next,time_gap_booking_prev_resort,time_gap_booking_next_resort,time_gap_checkin_prev_resort,time_gap_checkin_next_resort,time_gap_booking_prev2,time_gap_booking_next2,time_gap_checkin_prev2,time_gap_checkin_next2,prev_diff_days_stay,next_diff_days_stay,prev_diff_roomnights,next_diff_roomnights,prev_diff_days_advance_booking,next_diff_days_advance_booking,prev_diff_channel_code,next_diff_channel_code,prev_diff_room_type_booked_code,next_diff_room_type_booked_code,prev_diff_resort_type_code,next_diff_resort_type_code,prev_diff_main_product_code,next_diff_main_product_code,0b918943df0962bc7a1824c0555a389347b4febdc7cf9d1254406d80ce44e3f9,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac8f9e9a2596173f80c9,3e1e967e9b793e908f8eae83c74dba9bcccce6a5535b4b462bd9994537bfe15c,3fdba35f04dc8c462986c992bcf875546257113072a909c162f7e470e581e278,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c111b49baf362511afc,49d180ecf56132819571bf39d9b7b342522a2ac6d23c1418d3338251bfe469c8,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a,4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce,4ec9599fc203d176a301536c2e091a19bc852759b255bd6818810a42c5fed14a,535fa30d7e25dd8a49f1536779734ec8286108d115da5045d77f3b4185d8f790,6208ef0f7750c111548cf90b6ea1d0d0a66f6bff40dbef07cb45ec436263c7d6,624b60c58c9d8bfb6ff1886c2fd605d2adeb6ea4da576068201b6c6958ce93f4,670671cd97404156226e507973f2ab8330d3022ca96e0c93bdbdb320c41adcaf,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b,7902699be42c8a8e46fbbb4501726517e86b22c56a189f7625a6da49081b2451,7f2253d7e228b22a08bda1f09c516f6fead81df6536eb02fa991a34bb38d9be8,81b8a03f97e8787c53fe1a86bda042b6f0de9b0ec9c09357e107c99ba4d6948a,8722616204217eddb39e7df969e0698aed8e599ba62ed2de1ce49b03ade0fede,9400f1b21cb527d7fa3d3eabba93557a18ebe7a2ca4e471cfe5e4c5b4ca7f767,98a3ab7c340e8a033e7b37b6ef9428751581760af67bbab2b9e05d4964a8874a,9f14025af0065b30e47e23ebb3b491d39ae8ed17d33739e5ff3827ffb3634953,a68b412c4282555f15546cf6e1fc42893b7e07f271557ceb021821098dd66c1b,b17ef6d19c7a5b1ee83b907c595526dcb1eb06db8227d650d5dda0a9f4ce8cd9,c6f3ac57944a531490cd39902d0f777715fd005efac9a30622d5f5205e7f6894,c75cb66ae28d8ebc6eded002c28a8ba0d06d3a78c6b5cbf9b2ade051f0775ac4,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35,da4ea2a5506f2693eae190d9360a1f31793c98a1adade51d93533a6f520ace1c,e29c9c180c6279b0b02abd6a1801c7c04082cf486ec027aa13515e4f3884bb6b,e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683,ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d,f5ca38f748a1d6eaf726b8a42fb575c3c71f1864a8143301782de13da2d9202b,ff5a1ae012afa5d4c889c50ad427aaf545d31a4fac04ffc1c4d03d403ba4250a,2012,2015,2016,2017,2018,2019,0,1_x,2_x,3_x,4_x,5_x,7,1_y,2_y,3_y,4_y,5_y,6
0,821a5e5313029b1bbec6ec738a1b34d6d14c82fca97c47...,1.487470e+09,1.472798e+09,2.545455,-169.818182,2.545455,28,28,9,0,3,3,NaN,101088000.0,NaN,-90633600.0,NaN,-90547200.0,NaN,8294400.0,NaN,-185155200.0,NaN,84326400.0,NaN,-113184000.0,NaN,1.0,NaN,-1.0,NaN,-2219.0,0,1,0,0,0,1,0,1,0,0,1,0,0,3,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,1,3,0,3,4,0,0,6,1,3,0,0,1,0,5,2,4,0,0
1,a80d2fbea1195021307a13a44642855c60f46145dcf29c...,1.522656e+09,1.459642e+09,3.000000,-729.333333,1.666667,9,5,2,0,6,3,NaN,0.0,NaN,-192067200.0,NaN,-191721600.0,NaN,0.0,NaN,-192067200.0,NaN,-604800.0,NaN,-192326400.0,NaN,4.0,NaN,0.0,NaN,-2223.0,0,1,0,1,0,1,0,1,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,2,0,0,0,1,0,0,1,2,0,0,0
2,548605b68da37dd5c8555a37650910ce9f110e793b4619...,1.522886e+09,1.427630e+09,3.500000,

In [19]:
new_df.to_csv("feat_v2.csv", index=False)